In [4]:
import mindspore.ops as ops

def clip_by_norm(clip_norm, t, axis=None):
    """给定张量t和裁剪参数clip_norm，对t进行正则化

    使得t在axes维度上的L2-norm小于等于clip_norm。

    Args:
        t: tensor，数据类型为float
        clip_norm: scalar，数值需大于0；梯度裁剪阈值，数据类型为float
        axis: Union[None, int, tuple(int)]，数据类型为int32；计算L2-norm参考的维度，如为Norm，则参考所有维度
    """

    # 计算L2-norm
    t2 =  t * t
    l2sum = t2.sum(axis=axis, keepdims=True)
    pred = l2sum > 0
    # 将加和中等于0的元素替换为1，避免后续出现NaN
    l2sum_safe = ops.select(pred, l2sum, ops.ones_like(l2sum))
    l2norm = ops.select(pred, ops.sqrt(l2sum_safe), l2sum)
    # 比较L2-norm和clip_norm，如L2-norm超过阈值，进行裁剪
    # output(x) = (x * clip_norm)/max(|x|, clip_norm)
    intermediate = t * clip_norm
    cond = l2norm > clip_norm
    t_clip = intermediate / ops.select(cond, l2norm, clip_norm)

    return t_clip

In [5]:
import mindspore
from mindspore import Tensor

clip = 1.0
t = Tensor([1, 2, 3, 4], mindspore.float32)
clip_by_norm(clip, t)

<class 'mindspore.common.tensor.Tensor'>


Tensor(shape=[4], dtype=Float32, value= [ 1.82574183e-01,  3.65148365e-01,  5.47722518e-01,  7.30296731e-01])

In [34]:
a = Tensor([True, True, False])
a.shape[0]

3

In [36]:
# a = a.astype(mindspore.float32).reshape((-1,))
# print(a)
a = a.reshape((-1,))
b = a.sum().asnumpy()
print(type(b))
print(b)
print(b + 0.0)
print(f'{b: 5.3f}')

<class 'numpy.ndarray'>
2
2.0
 2.000


In [33]:
a = [1, 2, 4, 5, 6, 7]
b = [2, 3, 4, 5]
print(Tensor(a) == Tensor(b))

[False False  True  True]
